In [1]:
# %pip install transformers datasets torch
# %pip install pandas pyarrow


In [2]:
import pandas as pd

# Load the training and validation datasets
train_df = pd.read_parquet("train-00000-of-00001.parquet")
val_df = pd.read_parquet("validation-00000-of-00001.parquet")

# Check the structure
# print(train_df.head())


In [3]:
from datasets import Dataset

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Check sample data
print(train_dataset[0])


{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'answer_start': [269], 'text': ['in the late 1990s']}}


In [4]:
# print(train_dataset[0]['id'])


In [5]:
from transformers import AutoTokenizer

model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
train_dataset["answers"][0]["text"]

['in the late 1990s']

In [7]:
# Define the preprocess function to tokenize the data
def preprocess_function(examples):
    # Tokenize the question and context
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=384,
        truncation=True,
        padding="max_length",
        return_offsets_mapping=True
    )
    # print(examples["answers"][0]["answer_start"])
    # print(examples["context"])
    # return
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(inputs["offset_mapping"]):
        # If answers exist, map them to token positions
        if len(examples["answers"][i]["text"]) == 0:  # Handle no answer case
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Extract the start and end character positions of the answer
            start_char = examples["answers"][i]["answer_start"][0]
            end_char = start_char + len(examples["answers"][i]["text"][0])

            # Map character positions to token positions
            start_token = next((idx for idx, offset in enumerate(offsets) if offset[0] == start_char), 0)
            end_token = next((idx for idx, offset in enumerate(offsets) if offset[1] == end_char), 0)

            start_positions.append(start_token)
            end_positions.append(end_token)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs.pop("offset_mapping")  # Remove offset mapping after use to save memory

    return inputs


# Apply the preprocessing function to the datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Check the tokenized output (for the first example)
# print(train_dataset[0])


Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'answer_start': [269], 'text': ['in the late 1990s']}, 'input_ids': [128000, 4599, 1550, 54992, 13486, 1212, 10671, 5526, 30, 128000, 33, 1216, 95710, 480, 285, 6853

### Moddel Finetunning For Q&A: SQuAD

In [1]:
import torch
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# Load the model
model_name = "meta-llama/Llama-3.2-1B"  
model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Mixed precision
    device_map="auto"           # Distributes the model across available GPUs
)


KeyboardInterrupt: 

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",           # Directory for saving results
    evaluation_strategy="epoch",     # Evaluate after each epoch
    save_strategy="epoch",           # Save model after each epoch
    learning_rate=3e-4,              # Learning rate
    per_device_train_batch_size=16,  # Training batch size
    per_device_eval_batch_size=16,   # Evaluation batch size
    num_train_epochs=3,              # Number of epochs
    weight_decay=0.01,               # Weight decay
    logging_dir="./logs",            # Directory for logs
    logging_steps=10,                # Log every 10 steps
    save_total_limit=2,              # Save only the 2 most recent checkpoints
    load_best_model_at_end=True      # Load the best model after training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


In [ ]:
model.save_pretrained("./squad_fine_tuned_model")
tokenizer.save_pretrained("./squad_fine_tuned_model")


In [ ]:
from transformers import pipeline

# Load the pipeline for question answering
qa_pipeline = pipeline("question-answering", model="./squad_fine_tuned_model", tokenizer=tokenizer)

# Test on new examples
context = "The Eiffel Tower is located in Paris and is one of the most famous landmarks in the world."
question = "Where is the Eiffel Tower located?"

result = qa_pipeline({"context": context, "question": question})
print(result)
